# MySQL

In [ ]:
import datetime
import pymysql
pymysql.install_as_MySQLdb()

In [ ]:
# 1. DB 연결
def connect_db():
    conn = pymysql.connect(host="localhost", user="root", password="4team123!",
                           db="INVESTAR", charset="utf8", autocommit=True)  
    return conn

In [ ]:
conn = connect_db()
curs = conn.cursor() 
curs.execute("SELECT VERSION();")
result = curs.fetchone()
print("MySQL DB version : {}".format(result))

In [ ]:
# sql = "DROP TABLE company_info"
# curs.execute(sql)

sql = "DROP TABLE daily_price"
curs.execute(sql)

In [ ]:
sql = "SELECT DISTINCT code, company FROM company_info"
curs.execute(sql)
# stock_list = [item[0] for item in curs.fetchall()] 
# print(len(stock_list))

In [ ]:
datas = cur.fetchall()

In [ ]:
for data in datas:
    print(data)

In [ ]:
conn.commit()

In [ ]:
# 2. DB 연결 종료
conn.close()

In [ ]:
from Investar import Analyzer
mk = Analyzer.MarketDB()
mk.get_daily_price('삼성전자', '2019-09-30', '2021-07-03')
today = datetime.datetime.now().strftime('%Y%m%d') 

# Company Info

In [ ]:
import FinanceDataReader as fdr
import pandas as pd

"""KRX로부터 상장기업 목록 파일을 읽어와서 데이터프레임으로 반환"""
url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method='\
    'download&searchType=13'
krx = pd.read_html(url, header=0)[0]
krx = krx[['종목코드', '회사명']]
krx = krx.rename(columns={'종목코드': 'code', '회사명': 'company'})
krx.code = krx.code.map('{:06d}'.format)
krx['industry'] = ""
krx['industryCode'] = ""

In [ ]:
stock = fdr.StockListing('nasdaq')
stock = stock[['Symbol', 'Name', 'Industry', 'IndustryCode']]
stock = stock.rename(columns={'Symbol': 'code', 'Name': 'company', 'Industry': 'industry', 'IndustryCode': 'industryCode'})
stock.head()

In [ ]:
stock[stock['industryCode'] == '533010']

In [ ]:
df = stock

In [ ]:
df = df.replace("\'","\'\'", regex=True)

In [ ]:
df[df['industryCode'] == '533010']

In [ ]:
df = pd.concat([krx, stock])

In [ ]:
df = df.drop_duplicates('company', keep='last')

In [ ]:
df.describe()

# Daily Price

In [ ]:
import FinanceDataReader as fdr
import pandas as pd
from bs4 import BeautifulSoup
import pymysql, calendar, time, json
import requests
from datetime import datetime
from threading import Timer

code = '005930'
url = f"http://finance.naver.com/item/sise_day.nhn?code={code}"
print (url)

pages_to_fetch = 2

In [ ]:
stock = fdr.StockListing('nasdaq')

In [ ]:
%%writefile config.json
{"pages_to_fetch": 27}

# 최종 코드

In [4]:
%%writefile Investar/StockUpdater.py

import concurrent.futures
import FinanceDataReader as fdr
import math
import pandas as pd
import pymysql, calendar, time, json
import requests
from tqdm import tqdm

from bs4 import BeautifulSoup
from datetime import datetime
from dateutil.relativedelta import relativedelta
from threading import Timer

try:
    with open('config.json', 'r') as in_file:
        config = json.load(in_file)
        pages_to_fetch = config['pages_to_fetch']
except FileNotFoundError:
    with open('config.json', 'w') as out_file:
        pages_to_fetch = 100 
        config = {'pages_to_fetch': 1}
        json.dump(config, out_file)
            
class DBUpdater:
                
    def __init__(self):
        """생성자: MariaDB 연결 및 종목코드 딕셔너리 생성"""
        self.conn = pymysql.connect(host='localhost', user='root',
            password='4team123!', db='INVESTAR', charset='utf8')
        
        with self.conn.cursor() as curs:
            sql = """
            CREATE TABLE IF NOT EXISTS company_info (
                code VARCHAR(20),
                company VARCHAR(80),
                industry VARCHAR(80),
                industryCode VARCHAR(10),                
                last_update DATE,
                PRIMARY KEY (code))
            """
            curs.execute(sql)
            sql = """
            CREATE TABLE IF NOT EXISTS daily_price (
                code VARCHAR(20),
                date DATE,
                open BIGINT(20),
                high BIGINT(20),
                low BIGINT(20),
                close BIGINT(20),
                diff BIGINT(20),
                volume BIGINT(20),
                PRIMARY KEY (code, date))
            """
            curs.execute(sql)
        self.conn.commit()
        self.codes = dict()
               
    def __del__(self):
        """소멸자: MariaDB 연결 해제"""
        self.conn.close() 
        
    def read_ticker(self):
        """KRX로부터 상장기업 목록 파일을 읽어와서 데이터프레임으로 반환"""
        url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method='\
            'download&searchType=13'
        krx = pd.read_html(url, header=0)[0]
        krx = krx[['종목코드', '회사명']]
        krx = krx.rename(columns={'종목코드': 'code', '회사명': 'company'})
        krx.code = krx.code.map('{:06d}'.format)
        krx['industry'] = ""
        krx['industryCode'] = ""
        stock = fdr.StockListing('NASDAQ')
        stock = stock[['Symbol', 'Name', 'Industry', 'IndustryCode']]
        stock = stock.rename(columns={'Symbol': 'code', 'Name': 'company', 'Industry': 'industry', 'IndustryCode': 'industryCode'})
        stock = stock.replace("\'","\'\'", regex=True)
        df = pd.concat([krx, stock])
        df = df.rename(
            columns={'Symbol': 'code', 'Name': 'company', 'Industry': 'industry', 'IndustryCode': 'industryCode'})
        return df
    
    def update_comp_info(self):
        """종목코드를 company_info 테이블에 업데이트 한 후 딕셔너리에 저장"""
        sql = "SELECT * FROM company_info"
        df = pd.read_sql(sql, self.conn)
        for idx in range(len(df)):
            self.codes[df['code'].values[idx]] = df['company'].values[idx]
                    
        with self.conn.cursor() as curs:
            sql = "SELECT max(last_update) FROM company_info"
            curs.execute(sql)
            rs = curs.fetchone()
            today = datetime.today().strftime('%Y-%m-%d')
            if rs[0] == None or rs[0].strftime('%Y-%m-%d') < today:
                df = self.read_ticker()
                for idx in range(len(df)):
                    code = df.code.values[idx]
                    company = df.company.values[idx]
                    industry = df.industry.values[idx]
                    industryCode = df.industryCode.values[idx]
                    sql = f"REPLACE INTO company_info (code, company, industry, industryCode, last"\
                        f"_update) VALUES ('{code}', '{company}', '{industry}', '{industryCode}', '{today}')"
                    curs.execute(sql)
                    self.codes[code] = company
                    tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
                    print(f"[{tmnow}] #{idx+1:04d} REPLACE INTO company_info "\
                        f"VALUES ({code}, {company}, {industry}, {industryCode}, {today})")
                self.conn.commit()
                print('')              

    def read_naver_finance(self, code):
        """네이버에서 주식 시세를 읽어서 데이터프레임으로 반환"""
        try:
            url = f"http://finance.naver.com/item/sise_day.nhn?code={code}"
            html = BeautifulSoup(requests.get(url,
                headers={'User-agent': 'Mozilla/5.0'}).text, "lxml")
            pgrr = html.find("td", class_="pgRR")
            if pgrr is None:
                return None
            s = str(pgrr.a["href"]).split('=')
            lastpage = s[-1] 
            krx = pd.DataFrame()
            pages = min(int(lastpage), pages_to_fetch)
            for page in range(1, pages + 1):
                pg_url = '{}&page={}'.format(url, page)
                krx = krx.append(pd.read_html(requests.get(pg_url,
                    headers={'User-agent': 'Mozilla/5.0'}).text)[0])                                          
                tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
#                 print('[{}] {} ({}) : {:04d}/{:04d} pages are downloading...'.
#                     format(tmnow, company, code, page, pages), end="\r")
            krx = krx.rename(columns={'날짜':'date','종가':'close','전일비':'diff'
                ,'시가':'open','고가':'high','저가':'low','거래량':'volume'})
            krx['date'] = krx['date'].replace('.', '-')
            krx['code'] = code
            krx = krx.dropna()
            krx[['close', 'diff', 'open', 'high', 'low', 'volume']] = krx[['close',
                'diff', 'open', 'high', 'low', 'volume']].astype(int)
            krx = krx[['date', 'code','open', 'high', 'low', 'close', 'diff', 'volume']]
            krx = krx.reset_index(drop=True)
            df = krx.sort_values(by='date', ascending=False)

        except Exception as e:
            print('Exception occured :', str(e))
            return None
        return df

    def read_yahoo_finance(self, code):
        """야후 파이낸스에서 주식 시세를 가져옴(데이터플레임)"""
        try:      
            now = datetime.now()
            months = math.ceil( (pages_to_fetch * 10) / (262 / 12) )
            start_date = now - relativedelta(months=months)
            stock = fdr.DataReader(code, start_date.strftime('%Y-%m-%d'))
            days_ago = 10 * pages_to_fetch
            stock = stock[-(days_ago+1):]
            stock['code'] = code
            stock['diff'] = abs( stock['Close'] - stock['Close'].shift(1))
            stock = stock.reset_index()[-days_ago:].sort_values(by='Date', ascending=False)
            stock = stock.rename(columns={'Date':'date','Close':'close','diff':'diff'
                ,'Open':'open','High':'high','Low':'low','Volume':'volume'})
            stock = stock[['date','code','open','high','low','close','diff','volume']]
            df = stock.reset_index(drop=True)
            df.dropna(inplace=True)
        except Exception as e:
            return None
        return stock
    
    def replace_into_db(self, df):
        """네이버에서 읽어온 주식 시세를 DB에 REPLACE"""
        with self.conn.cursor() as curs:
            for r in df.itertuples():
                sql = f"REPLACE INTO daily_price VALUES ('{r.code}', "\
                    f"'{r.date}', {r.open}, {r.high}, {r.low}, {r.close}, "\
                    f"{r.diff}, {r.volume})"
                curs.execute(sql)
            self.conn.commit()
            print('completed')
#             print('[{}] #{:04d} {} ({}) : {} rows > REPLACE INTO daily_'\
#                 'price [OK]'.format(datetime.now().strftime('%Y-%m-%d'\
#                 ' %H:%M'), num+1, company, code, len(df)))

            
    def replace_into_db(self, df):
        """네이버에서 읽어온 주식 시세를 DB에 REPLACE"""
        with self.conn.cursor() as curs:
            for r in df.itertuples():
                sql = f"REPLACE INTO daily_price VALUES ('{r.code}', "\
                    f"'{r.date}', {r.open}, {r.high}, {r.low}, {r.close}, "\
                    f"{r.diff}, {r.volume})"
                curs.execute(sql)
            self.conn.commit()
            print("daily_price updated")
#             print('[{}] #{:04d} {} ({}) : {} rows > REPLACE INTO daily_'\
#                 'price [OK]'.format(datetime.now().strftime('%Y-%m-%d'\
#                 ' %H:%M'), df.count(), {r.company}, {r.code}, len(df)))   
            
    def update_daily_naver_price(self, codes):
        MAX_THREADS = 300
        threads = min(MAX_THREADS, len(codes))
        df = pd.DataFrame()
        with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
            for number, stock in tqdm(zip(codes, executor.map(self.read_naver_finance, codes))):
                df = df.append(stock)
        df = df.reset_index(drop=True)
        df.dropna(inplace=True) 
        return df

    def update_daily_yahoo_price(self, codes):
        MAX_THREADS = 300
        threads = min(MAX_THREADS, len(codes))
        df = pd.DataFrame()
        with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
            for number, stock in tqdm(zip(codes, executor.map(self.read_yahoo_finance, codes))):
                df = df.append(stock)
        df = df.reset_index(drop=True)
        df.dropna(inplace=True) 
        return df
    
    def execute_daily(self):
        """실행 즉시 및 매일 오후 다섯시에 daily_price 테이블 업데이트"""
        self.update_comp_info()
                
        list_nf = []; list_yf = []
        for code in self.codes:
            if code.isdigit() == True:
                list_nf.append(code)
            else:
                list_yf.append(code)

        codes = list_nf
        df = self.update_daily_naver_price(codes)
        self.replace_into_db(df)  

        codes = list_yf
        df = self.update_daily_yahoo_price(codes)
        self.replace_into_db(df)  

        tmnow = datetime.now()
        lastday = calendar.monthrange(tmnow.year, tmnow.month)[1]
        if tmnow.month == 12 and tmnow.day == lastday:
            tmnext = tmnow.replace(year=tmnow.year+1, month=1, day=1,
                hour=17, minute=0, second=0)
        elif tmnow.day == lastday:
            tmnext = tmnow.replace(month=tmnow.month+1, day=1, hour=17,
                minute=0, second=0)
        else:
            tmnext = tmnow.replace(day=tmnow.day+1, hour=17, minute=0,
                second=0)   
        tmdiff = tmnext - tmnow
        secs = tmdiff.seconds
        t = Timer(secs, self.execute_daily)
        print("Waiting for next update ({}) ... ".format(tmnext.strftime
            ('%Y-%m-%d %H:%M')))
        t.start()
    
if __name__ == '__main__':
    dbu = DBUpdater()
    dbu.execute_daily()


Overwriting Investar/StockUpdater.py


# DB 적재

In [1]:
from Investar import StockUpdater

In [2]:
dbu = StockUpdater.DBUpdater()

In [3]:
df = dbu.execute_daily()

1802it [25:51,  3.39s/it]

Exception occured :Exception occured : ('Connection aborted.', TimeoutError(10060, '연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다', None, 10060, None))
 ('Connection aborted.', TimeoutError(10060, '연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다', None, 10060, None))


2451it [33:33,  1.22it/s]


NameError: name 'num' is not defined

In [ ]:
297 - 08:20
600 - 08:23
901 - 08:29
1504 - 08:39
2451 - 08:57

In [4]:
df

""


In [ ]:
# dbu.read_yahoo_finance()
# dbu.update_comp_info()

In [1]:
import concurrent.futures
import FinanceDataReader as fdr
import math
import pandas as pd
import pymysql, calendar, time, json
import requests
import tqdm

from bs4 import BeautifulSoup
from datetime import datetime
from dateutil.relativedelta import relativedelta
from threading import Timer

MAX_THREADS = 300
pages_to_fetch = 27

codes = ['AACG', 'AAL', 'AAME', 'AAOI', 'AAON', 'AAPL', 'AAWW', 'ABCB']

def read_yahoo_finance(code):
    """야후 파이낸스에서 주식 시세를 가져옴(데이터플레임)"""
    try:      
        now = datetime.now()
        months = math.ceil( (pages_to_fetch * 10) / (262 / 12) )
        start_date = now - relativedelta(months=months)
        stock = fdr.DataReader(code, start_date.strftime('%Y-%m-%d'))
        days_ago = 10 * pages_to_fetch
        stock = stock[-(days_ago+1):]
        stock['code'] = code
        stock['diff'] = abs( stock['Close'] - stock['Close'].shift(1))
        stock = stock.reset_index()[-days_ago:].sort_values(by='Date', ascending=False)
        stock = stock.rename(columns={'Date':'date','Close':'close','diff':'diff'
            ,'Open':'open','High':'high','Low':'low','Volume':'volume'})
        stock = stock[['date','code','open','high','low','close','diff','volume']]
        df = stock.reset_index(drop=True)
        df.dropna(inplace=True)
        print(stock)
    except Exception as e:
        print('Exception occured :', str(e))
        return None
    return stock


def main():
    threads = min(MAX_THREADS, len(codes))
    df = pd.DataFrame()
    with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
        for code, stock in zip(codes, executor.map(read_yahoo_finance, codes)):
            print('%s : %s' % (code, stock))
            df = df.append(stock)
    df.dropna(inplace=True) 
    return df


if __name__ == '__main__':
    main()

Exception occured : Expecting value: line 1 column 1 (char 0)
Exception occured : Expecting value: line 1 column 1 (char 0)
Exception occured : Expecting value: line 1 column 1 (char 0)
Exception occured : Expecting value: line 1 column 1 (char 0)
Exception occured : Expecting value: line 1 column 1 (char 0)Exception occured :Exception occured : Expecting value: line 1 column 1 (char 0)

 Expecting value: line 1 column 1 (char 0)
Exception occured : Expecting value: line 1 column 1 (char 0)AACG : None

AAL : None
AAME : None
AAOI : None
AAON : None
AAPL : None
AAWW : None
ABCB : None


In [ ]:
    with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
        for number, df in tqdm(zip(codes, executor.map(read_yahoo_finance, codes))):